### Libraries

In [1]:
'generic imports'
import os
import pandas as pd
import sys
sys.path.append(os.path.abspath('..'))
from src import utils
import numpy as np
import datetime

'machine learning library imports'
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

### Load data

In [2]:
data_dir = os.path.abspath('../data')

# Non-augmented dataset
df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k.csv'), low_memory=False)
AUGMENTATION = 'None'

# SMOTE augmented dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_SMOTE.csv'), low_memory=False)
# AUGMENTATION = 'SMOTE'

# SMOTE-NC augmented dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_SMOTE_NC.csv'), low_memory=False)
# AUGMENTATION = 'SMOTE-NC'

# RealTabFormer augmentation dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_RealTabFormer.csv'), low_memory=False)
# AUGMENTATION = 'RealTabFormer'

# GReaT augmentation dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_GReaT.csv'), low_memory=False)
# AUGMENTATION = 'GReaT'

# Test data for all datasets
df_test = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_test.csv'), low_memory=False)

### Data preparation

In [3]:
# Drop columns mbtcp.unit_id and mbtcp.trans_id from train and test data    
df_train = df_train.drop(['mbtcp.unit_id', 'mbtcp.trans_id'], axis=1)
df_test = df_test.drop(['mbtcp.unit_id', 'mbtcp.trans_id'], axis=1)

# Creates X_train, y_train
X_train = df_train.drop(['Attack_label', 'Attack_type'], axis=1)
y_train = df_train['Attack_type']

# Creates X_test, y_test
X_test = df_test.drop(['Attack_label', 'Attack_type'], axis=1)
y_test = df_test['Attack_type']

#### Convert categorical features to one-hot encoded features

In [4]:
# Extract categorical features
categorical_features = X_train.select_dtypes(include="object").columns

# Concatenate X_train and X_test
X_comb = pd.concat([X_train[categorical_features], X_test[categorical_features]], axis=0)

# Apply one-hot encoding (get_dummies)
X_comb_enc = pd.get_dummies(X_comb, dtype='int8')

# Split back into X_train and X_test
X_train_enc, X_test_enc = train_test_split(
    X_comb_enc, test_size=len(X_test), random_state=42)

# Print the shape of X_train_enc and X_test_enc
print(f'X_train_enc shape: {X_train_enc.shape}, X_test_enc shape: {X_test_enc.shape}')

X_train_enc shape: (536515, 53), X_test_enc shape: (381934, 53)


In [5]:
# Drop columns categorical_features from X_train and X_test 
X_train = X_train.drop(categorical_features, axis=1)
X_test = X_test.drop(categorical_features, axis=1)

# Concatenate X_train and X_train_enc
X_train = pd.concat([X_train.reset_index(drop=True), X_train_enc.reset_index(drop=True)], axis=1)
X_test = pd.concat([X_test.reset_index(drop=True), X_test_enc.reset_index(drop=True)], axis=1)

# Print the shape of X_train and X_test
print(f'X_train shape: {X_train.shape}, X_test shape: {X_test.shape}')

X_train shape: (536515, 92), X_test shape: (381934, 92)


#### Label encoding

In [6]:
# instantiate the label encoder
le = LabelEncoder()

# fit and encode the training labels
y_train = le.fit_transform(y_train)

# encode the test labels
y_test = le.transform(y_test)

print('Attack_type and encoded labels:\n')
for i, label in enumerate(le.classes_):
    print(f'{label:23s} {i:d}')

Attack_type and encoded labels:

Backdoor                0
DDoS_HTTP               1
DDoS_ICMP               2
DDoS_TCP                3
DDoS_UDP                4
Fingerprinting          5
MITM                    6
Normal                  7
Password                8
Port_Scanning           9
Ransomware              10
SQL_injection           11
Uploading               12
Vulnerability_scanner   13
XSS                     14


### Model training

In [7]:
# Shuffle training data
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Instantiate model with 10 decision trees
RF = RandomForestClassifier(n_estimators = 10, random_state = 42)

# Train the model
RF_clf =  xgb.fit(X_train, y_train)

### Model Evaluation

In [8]:
predictions = RF_clf.predict(X_test)

In [9]:
accuracy = metrics.accuracy_score(y_test, predictions)
precision = metrics.precision_score(y_test, predictions, average='weighted', zero_division=1)
recall = metrics.recall_score(y_test, predictions, average='weighted')
f1_score = metrics.f1_score(y_test, predictions, average='weighted')

print("Model Evaluation Metrics")
print("~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Accuracy: {:.2f}".format(accuracy))
print("Precision (Weighted): {:.2f}".format(precision))
print("Recall (Weighted): {:.2f}".format(recall))
print("F1(Weighted): {:.2f}".format(f1_score))
print("~~~~~~~~~~~~~~~~~~~~~~~~~")

Model Evaluation Metrics
~~~~~~~~~~~~~~~~~~~~~~~~~
Accuracy: 0.44
Precision (Weighted): 0.89
Recall (Weighted): 0.44
F1(Weighted): 0.49
~~~~~~~~~~~~~~~~~~~~~~~~~


#### Save Metrics Results 

In [10]:
# create dictionary for results
results = {
    "model": "Decision Tree",
    "augmentations": AUGMENTATION,
    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "accuracy": accuracy,
    "precision": precision,
    "recall": recall,
    "f1": f1_score
    }

# save results to csv   
utils.save_results_to_csv([results], '../results/metrics/dt.csv')

#### Confusion Matrix

In [11]:
conf_mat = metrics.confusion_matrix(y_test, predictions)

attack_labels = ['Backdoor', 'DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP', 
'Fingerprinting', 'MITM', 'Normal', 'Password', 'Port_Scanning', 'Ransomware', 
'SQL_injection', 'Uploading', 'Vulnerability_scanner', 'XSS']

# Create a dataframe from the confusion matrix
conf_mat_df = pd.DataFrame(conf_mat, 
                            index = attack_labels, 
                            columns = attack_labels)
conf_mat_df.index.name = 'Actual'
conf_mat_df.columns.name = 'Predicted'

# Save the confusion matrix
conf_mat_df.to_csv(f"../results/conf_matrix/{results['model']}_{results['augmentations']}.csv")
conf_mat_df

Predicted,Backdoor,DDoS_HTTP,DDoS_ICMP,DDoS_TCP,DDoS_UDP,Fingerprinting,MITM,Normal,Password,Port_Scanning,Ransomware,SQL_injection,Uploading,Vulnerability_scanner,XSS
Actual,,,,,,,,,,,,,,,
Backdoor,4435,110,0,0,0,1,0,0,16,73,6,14,115,12,0
DDoS_HTTP,36,7634,0,0,1,1,0,0,537,0,12,740,531,44,92
DDoS_ICMP,0,0,13499,0,0,1,0,0,1,0,0,0,0,0,0
DDoS_TCP,0,0,0,10009,0,0,0,0,0,0,0,0,0,0,0
DDoS_UDP,0,0,0,0,24601,0,0,0,0,0,0,0,0,0,0
Fingerprinting,0,15,4,0,0,94,0,0,2,23,0,2,4,0,2
MITM,0,0,0,0,0,1,75,0,0,0,0,0,0,0,0
Normal,16206,37986,1,1,61,13,8,71479,101875,0,1926,14173,22055,5826,1166
Password,6,667,0,0,0,2,0,0,8276,0,1,759,308,16,73
